# Prepare Mosaic use case from NeurIPS competition data
Author: Pia Rautenstrauch

Date: 4th of February, 2023

- Rationale: use only a single paired data set, three unpaired data sets, as paired assays continue to be more expensive and technically challenging, so it is more likely we only have a few paired data sets
- Donor 1 measured at s1, s2, s4

## Control
- s1: atac only 
- s2: gex only
- s3: atac only
- s4: paired

## A
- s1: atac only - subsampled atac
- s2: gex only
- s3: atac only - subsampled atac 
- s4: paired

## B
- s1: atac only 
- s2: gex only
- s3 atac only 
- s4: paired - subsampled atac 

In [1]:
# Imports
import liam_NeurIPS2021_challenge_reproducibility
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import anndata as ad
import os
import scvi

Global seed set to 0
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# Load data
ad_mod1 = ad.read_h5ad("./../../data/original/neurips_competition/openproblems_bmmc_multiome_phase2/openproblems_bmmc_multiome_phase2.censor_dataset.output_mod1.h5ad")
ad_mod2 = ad.read_h5ad("./../../data/original/neurips_competition/openproblems_bmmc_multiome_phase2/openproblems_bmmc_multiome_phase2.censor_dataset.output_mod2.h5ad")

In [3]:
ad_mod1.obs["sample"] = ad_mod1.obs["batch"]
ad_mod1.obs["donor"] = ad_mod1.obs["batch"].apply(lambda x: x.split("d")[1])
ad_mod1.obs["site"] = ad_mod1.obs["batch"].apply(lambda x: x.split("d")[0])

In [4]:
ad_mod2.obs["sample"] = ad_mod2.obs["batch"]
ad_mod2.obs["donor"] = ad_mod2.obs["batch"].apply(lambda x: x.split("d")[1])
ad_mod2.obs["site"] = ad_mod2.obs["batch"].apply(lambda x: x.split("d")[0])

In [5]:
ad_mod1.obs['site'].value_counts()

s4    20009
s1    15560
s2    13662
s3    13270
Name: site, dtype: int64

In [6]:
# Control 

In [7]:
ad_mod1.obs['mod'] = 'paired'

In [8]:
ad_mod1.write_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_full.output_mod1.h5ad")

/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'donor' as categorical
/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'site' as categorical
/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The 

In [9]:
ad_mod2.write_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_full.output_mod2.h5ad")

/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'donor' as categorical
/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'site' as categorical


In [7]:
# sites with d1 measured: s1, s2, s4

In [8]:
# s1 and s3: only atac

In [6]:
ad_mod1.obs['mod'] = ad_mod1.obs['site'].apply(lambda x: 'atac' if ((x == 's1') | (x == 's3')) else 'gex' if x == 's2' else 'paired')

In [9]:
ad_mod1.layers["counts"][(ad_mod1.obs['site'] == 's1') | (ad_mod1.obs['site'] == 's3')] = 0

/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [10]:
ad_mod1.layers["counts"][(ad_mod1.obs['site'] == 's1') | (ad_mod1.obs['site'] == 's3')].max()

0.0

In [11]:
ad_mod1.layers["counts"].max()

3027.0

In [12]:
# s2: only gex

In [13]:
ad_mod2.X[ad_mod2.obs['site'] == 's2'] = 0

In [14]:
ad_mod2.X[ad_mod2.obs['site'] == 's2'].max()

0.0

In [15]:
ad_mod2.X.max()

1.0

In [16]:
ad_mod1.obs['site'].value_counts()

s4    20009
s1    15560
s2    13662
s3    13270
Name: site, dtype: int64

In [17]:
ad_mod1.obs['mod'].value_counts()

atac      28830
paired    20009
gex       13662
Name: mod, dtype: int64

In [18]:
ad_mod1.write_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_a.output_mod1.h5ad")

/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'donor' as categorical
/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'site' as categorical
/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The 

In [19]:
ad_mod2.write_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_a.output_mod2.h5ad")

/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'donor' as categorical
/home/prauten/miniconda3/envs/dynamic_LIAM_challenge_reproducibility/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'site' as categorical


In [20]:
ad_mod1.write_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_b.output_mod1.h5ad")

In [21]:
ad_mod2.write_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_b.output_mod2.h5ad")

# Downsample ATAC modality by dropping non-zero (binary) observations 

Aim: Investigate how a low quality/coverage modality impacts joint vs. concatenated models

This is a heuristic strategy mimicing lower coverage data. If one had indeed measured a lower coverage dataset, the features selected would differ due to differences in the signal to noise ratios. Here, we called features with full "high quality" dataset, and drop observations from that.

Randomly drop
- 90% 

of observations.

In [22]:
# Create multiple subsamples for 10% 

In [23]:
def which_observations_to_drop(indices, percent):
    """
    Helper function to drop certain percent of non-zero observations from a sparse array.
    
    Parameters
    ----------
    indices: list
        (sorted) list of column indices of non-zero elements (per cell)
    
    percent: float
        How many nonzero observations to drop (percent).
    
    Returns
    -------
    np.array
        Sorted array of nonzero indices to drop.
    """
    # Convert list to array
    arr = np.array(indices)
    # How many nonzero observations to drop - absolute
    drop = int(len(indices) * percent)
    # Randomly permute the observations
    np.random.shuffle(arr)
    # Return sorted array of indices of non-zero elements to drop
    return np.sort(arr[:drop])
    

In [24]:
seeds = [8831, 234, 11, 9631, 94]

In [25]:
# a
# Set random seed for reproducibility
full = ad.read_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_a.output_mod2.h5ad")
full.obs['num_index'] = np.arange(0,full.shape[0])

nonzero_index_list_per_cell = full.X.tolil().rows
# List of Lists Format (LIL) format
# each row is a Python list (sorted) of column indices of non-zero elements
# https://scipy-lectures.org/advanced/scipy_sparse/lil_matrix.html?highlight=lil

input_backup = full.copy()

## Subsampling to 10% of the data
### In other words dropping 90% of the observations
for seed in seeds:
    print(seed)
    np.random.seed(seed)

    full = input_backup.copy()

    # Iterate over cells from site 1 and site 3
    for cell in full.obs[(full.obs['site'] == 's1') | (full.obs['site'] == 's3')]['num_index'].values.tolist():
        drop = which_observations_to_drop(nonzero_index_list_per_cell[cell], 0.9)
        # Which elements to set to zero (inplace)
        full.X[cell, drop.tolist()] = 0

    full.write_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_a.output_mod2_10_subsample_seed_{}.h5ad".format(seed))


    sum_after = full.X.sum(axis=1)

    sum_before = input_backup.X.sum(axis=1)

    print(sum_before)
    print(sum_after)
    print()


8831
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[ 196.]
 [ 397.]
 [ 228.]
 ...
 [4216.]
 [8949.]
 [3157.]]

234
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[ 196.]
 [ 397.]
 [ 228.]
 ...
 [4216.]
 [8949.]
 [3157.]]

11
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[ 196.]
 [ 397.]
 [ 228.]
 ...
 [4216.]
 [8949.]
 [3157.]]

9631
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[ 196.]
 [ 397.]
 [ 228.]
 ...
 [4216.]
 [8949.]
 [3157.]]

94
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[ 196.]
 [ 397.]
 [ 228.]
 ...
 [4216.]
 [8949.]
 [3157.]]



In [26]:
# b
# Set random seed for reproducibility
full = ad.read_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_b.output_mod2.h5ad")
full.obs['num_index'] = np.arange(0,full.shape[0])

nonzero_index_list_per_cell = full.X.tolil().rows
# List of Lists Format (LIL) format
# each row is a Python list (sorted) of column indices of non-zero elements
# https://scipy-lectures.org/advanced/scipy_sparse/lil_matrix.html?highlight=lil

input_backup = full.copy()

## Subsampling to 10% of the data
### In other words dropping 90% of the observations
for seed in seeds:
    print(seed)
    np.random.seed(seed)

    full = input_backup.copy()

    # Iterate over cells from site 4
    for cell in full.obs[full.obs['site'] == 's4']['num_index'].values.tolist():
        drop = which_observations_to_drop(nonzero_index_list_per_cell[cell], 0.9)
        # Which elements to set to zero (inplace)
        full.X[cell, drop.tolist()] = 0

    full.write_h5ad("./../../data/derived/neurips_competition/mosaic_use_case_b.output_mod2_10_subsample_seed_{}.h5ad".format(seed))


    sum_after = full.X.sum(axis=1)

    sum_before = input_backup.X.sum(axis=1)

    print(sum_before)
    print(sum_after)
    print()


8831
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[1951.]
 [3965.]
 [2279.]
 ...
 [ 422.]
 [ 895.]
 [ 316.]]

234
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[1951.]
 [3965.]
 [2279.]
 ...
 [ 422.]
 [ 895.]
 [ 316.]]

11
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[1951.]
 [3965.]
 [2279.]
 ...
 [ 422.]
 [ 895.]
 [ 316.]]

9631
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[1951.]
 [3965.]
 [2279.]
 ...
 [ 422.]
 [ 895.]
 [ 316.]]

94
[[1951.]
 [3965.]
 [2279.]
 ...
 [4216.]
 [8949.]
 [3157.]]
[[1951.]
 [3965.]
 [2279.]
 ...
 [ 422.]
 [ 895.]
 [ 316.]]

